# Test poke-env api

You need to install poke-env api by
```{commandline}
$ pip install poke-env
```
The poke-env requirements does not include dataclasses module, so if error occurs importing poke_env, you need to install it manually.

```{commandline}
$ pip install dataclasses
```

Once that is complete, clone the pokemon-showdown implementation
```{commandline}
$ git clone https://github.com/hsahovic/Pokemon-Showdown.git
```


### Random Player

Create random players.
Before running the below code, you should be running your pokemon-showdown server on your localhost.

In [1]:
import asyncio

from poke_env.player.random_player import RandomPlayer
from poke_env.player.utils import cross_evaluate
from tabulate import tabulate
import tensorflow as tf

C:\Users\sebeo\anaconda3\envs\rl_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sebeo\anaconda3\envs\rl_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sebeo\anaconda3\envs\rl_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sebeo\anaconda3\envs\rl_env\lib\site-packages\tensorflow\pyth

Create 3 RandomPlayer agents and battle with each other 20 times each.

In [ ]:
players = [RandomPlayer(max_concurrent_battles=10, battle_format="gen4randombattle") for _ in range(3)]

In [ ]:
cross_evaluation = await cross_evaluate(players, n_challenges=20)

table = [["-"] + [p.username for p in players]]

for p_1, results in cross_evaluation.items():
    table.append([p_1] + [cross_evaluation[p_1][p_2] for p_2 in results])

print(tabulate(table))

### Max-damage player (Heuristic)

In [2]:
from poke_env.player.player import Player
from poke_env.environment.battle import Battle

Create a max damage player that chooses a move with maximum damage

In [51]:
class MaxDamagePlayer(Player):
    def choose_move(self, battle: Battle) -> str:
        curr_poke = battle.active_pokemon
        oppo_poke = battle.opponent_active_pokemon
        print(f"level: {curr_poke.level}")
        stat = curr_poke.stats
        for s in stat.keys():
            print(f"{s}:{stat[s]}")
        print(f"Ability: {curr_poke.ability}")
        print("Moves")
        print(curr_poke._last_request)
        moves = curr_poke.moves
        for m in moves:
            print(f"{m}: {moves[m]}")
        print(curr_poke.status)
        print(f"Effects: {list(curr_poke.effects)}")
        print(f"Item: {curr_poke.item}")
        boosts = curr_poke.boosts
        for b in boosts.keys():
            print(f"{b}: {boosts[b]}")
        
        print("="*10)
        # If the player can attack, it will
        if battle.available_moves:
            # Finds the best move among available ones
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            return self.create_order(best_move)
        # If no attack is available, a random switch will be made
        else:
            return self.choose_random_move(battle)

Cross evaluate with RandomPlayer

In [52]:
    random_player = RandomPlayer(battle_format="gen8randombattle")
    max_damage_player = MaxDamagePlayer(battle_format="gen8randombattle")

In [53]:
await max_damage_player.battle_against(random_player, n_battles=100)

print(f"Max damage player won {max_damage_player.n_won_battles} out of 100 battles")

level: 88
atk:235
def:189
spa:112
spd:244
spe:184
Ability: ironfist
Moves
{'ident': 'p1: Hitmonchan', 'details': 'Hitmonchan, L88, M', 'condition': '231/231', 'active': True, 'stats': {'atk': 235, 'def': 189, 'spa': 112, 'spd': 244, 'spe': 184}, 'moves': ['machpunch', 'rapidspin', 'throatchop', 'drainpunch'], 'baseAbility': 'ironfist', 'item': 'assaultvest', 'pokeball': 'pokeball', 'ability': 'ironfist'}
machpunch: machpunch (Move object)
rapidspin: rapidspin (Move object)
throatchop: throatchop (Move object)
drainpunch: drainpunch (Move object)
None
Effects: []
Item: assaultvest
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 88
atk:235
def:189
spa:112
spd:244
spe:184
Ability: ironfist
Moves
{'ident': 'p1: Hitmonchan', 'details': 'Hitmonchan, L88, M', 'condition': '231/231', 'active': True, 'stats': {'atk': 235, 'def': 189, 'spa': 112, 'spd': 244, 'spe': 184}, 'moves': ['machpunch', 'rapidspin', 'throatchop', 'drainpunch'], 'baseAbility': 'ironfist', 'item': 'assaultv

level: 88
atk:188
def:179
spa:184
spd:175
spe:156
Ability: oblivious
Moves
{'ident': 'p1: Whiscash', 'details': 'Whiscash, L88, F', 'condition': '59/337', 'active': True, 'stats': {'atk': 188, 'def': 179, 'spa': 184, 'spd': 175, 'spe': 156}, 'moves': ['liquidation', 'zenheadbutt', 'stoneedge', 'earthquake'], 'baseAbility': 'oblivious', 'item': 'choiceband', 'pokeball': 'pokeball', 'ability': 'oblivious'}
liquidation: liquidation (Move object)
zenheadbutt: zenheadbutt (Move object)
stoneedge: stoneedge (Move object)
earthquake: earthquake (Move object)
None
Effects: []
Item: choiceband
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 88
atk:188
def:179
spa:184
spd:175
spe:156
Ability: oblivious
Moves
{'ident': 'p1: Whiscash', 'details': 'Whiscash, L88, F', 'condition': '0 fnt', 'active': True, 'stats': {'atk': 188, 'def': 179, 'spa': 184, 'spd': 175, 'spe': 156}, 'moves': ['liquidation', 'zenheadbutt', 'stoneedge', 'earthquake'], 'baseAbility': 'oblivious', 'item': 'choi

level: 86
atk:116
def:152
spa:238
spd:213
spe:161
Ability: waterabsorb
Moves
{'ident': 'p1: Vaporeon', 'details': 'Vaporeon, L86, F', 'condition': '150/364', 'active': True, 'stats': {'atk': 116, 'def': 152, 'spa': 238, 'spd': 213, 'spe': 161}, 'moves': ['scald', 'wish', 'toxic', 'protect'], 'baseAbility': 'waterabsorb', 'item': 'leftovers', 'pokeball': 'pokeball', 'ability': 'waterabsorb'}
scald: scald (Move object)
wish: wish (Move object)
toxic: toxic (Move object)
protect: protect (Move object)
None
Effects: []
Item: leftovers
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: -1
spe: 0
level: 86
atk:116
def:152
spa:238
spd:213
spe:161
Ability: waterabsorb
Moves
{'ident': 'p1: Vaporeon', 'details': 'Vaporeon, L86, F', 'condition': '64/364', 'active': True, 'stats': {'atk': 116, 'def': 152, 'spa': 238, 'spd': 213, 'spe': 161}, 'moves': ['scald', 'wish', 'toxic', 'protect'], 'baseAbility': 'waterabsorb', 'item': 'leftovers', 'pokeball': 'pokeball', 'ability': 'waterabsorb'}
scald: scal

level: 80
atk:254
def:198
spa:174
spd:182
spe:209
Ability: roughskin
Moves
{'ident': 'p1: Garchomp', 'details': 'Garchomp, L80, F', 'condition': '304/304', 'active': True, 'stats': {'atk': 254, 'def': 198, 'spa': 174, 'spd': 182, 'spe': 209}, 'moves': ['outrage', 'firefang', 'swordsdance', 'earthquake'], 'baseAbility': 'roughskin', 'item': 'lumberry', 'pokeball': 'pokeball', 'ability': 'roughskin'}
outrage: outrage (Move object)
firefang: firefang (Move object)
swordsdance: swordsdance (Move object)
earthquake: earthquake (Move object)
None
Effects: []
Item: lumberry
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 80
atk:254
def:198
spa:174
spd:182
spe:209
Ability: roughskin
Moves
{'ident': 'p1: Garchomp', 'details': 'Garchomp, L80, F', 'condition': '304/304', 'active': True, 'stats': {'atk': 254, 'def': 198, 'spa': 174, 'spd': 182, 'spe': 209}, 'moves': ['outrage', 'firefang', 'swordsdance', 'earthquake'], 'baseAbility': 'roughskin', 'item': 'lumberry', 'pokeball': 'p

Ability: grassysurge
Moves
{'ident': 'p1: Tapu Bulu', 'details': 'Tapu Bulu, L82', 'condition': '201/249', 'active': True, 'stats': {'atk': 260, 'def': 236, 'spa': 187, 'spd': 203, 'spe': 170}, 'moves': ['megahorn', 'woodhammer', 'stoneedge', 'swordsdance'], 'baseAbility': 'grassysurge', 'item': 'lumberry', 'pokeball': 'pokeball', 'ability': 'grassysurge'}
megahorn: megahorn (Move object)
woodhammer: woodhammer (Move object)
stoneedge: stoneedge (Move object)
swordsdance: swordsdance (Move object)
None
Effects: []
Item: lumberry
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 82
atk:260
def:236
spa:187
spd:203
spe:170
Ability: grassysurge
Moves
{'ident': 'p1: Tapu Bulu', 'details': 'Tapu Bulu, L82', 'condition': '201/249', 'active': True, 'stats': {'atk': 260, 'def': 236, 'spa': 187, 'spd': 203, 'spe': 170}, 'moves': ['megahorn', 'woodhammer', 'stoneedge', 'swordsdance'], 'baseAbility': 'grassysurge', 'item': 'lumberry', 'pokeball': 'pokeball', 'ability': 'grassysurge'

...

level: 82
atk:139
def:183
spa:211
spd:211
spe:178
Ability: chlorophyll
Moves
{'ident': 'p1: Venusaur', 'details': 'Venusaur, L82, F', 'condition': '265/265', 'active': True, 'stats': {'atk': 139, 'def': 183, 'spa': 211, 'spd': 211, 'spe': 178}, 'moves': ['substitute', 'sludgebomb', 'sleeppowder', 'leechseed'], 'baseAbility': 'chlorophyll', 'item': 'blacksludge', 'pokeball': 'pokeball', 'ability': 'chlorophyll'}
substitute: substitute (Move object)
sludgebomb: sludgebomb (Move object)
sleeppowder: sleeppowder (Move object)
leechseed: leechseed (Move object)
None
Effects: []
Item: blacksludge
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 82
atk:139
def:183
spa:211
spd:211
spe:178
Ability: chlorophyll
Moves
{'ident': 'p1: Venusaur', 'details': 'Venusaur, L82, F', 'condition': '142/265', 'active': True, 'stats': {'atk': 139, 'def': 183, 'spa': 211, 'spd': 211, 'spe': 178}, 'moves': ['substitute', 'sludgebomb', 'sleeppowder', 'leechseed'], 'baseAbility': 'chlorophyll', 'i

spe:263
Ability: strongjaw
Moves
{'ident': 'p1: Boltund', 'details': 'Boltund, L88, F', 'condition': '213/265', 'active': True, 'stats': {'atk': 209, 'def': 156, 'spa': 209, 'spd': 156, 'spe': 263}, 'moves': ['bulkup', 'psychicfangs', 'thunderfang', 'firefang'], 'baseAbility': 'strongjaw', 'item': 'lifeorb', 'pokeball': 'pokeball', 'ability': 'strongjaw'}
bulkup: bulkup (Move object)
psychicfangs: psychicfangs (Move object)
thunderfang: thunderfang (Move object)
firefang: firefang (Move object)
None
Effects: []
Item: lifeorb
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 88
atk:209
def:156
spa:209
spd:156
spe:263
Ability: strongjaw
Moves
{'ident': 'p1: Boltund', 'details': 'Boltund, L88, F', 'condition': '187/265', 'active': True, 'stats': {'atk': 209, 'def': 156, 'spa': 209, 'spd': 156, 'spe': 263}, 'moves': ['bulkup', 'psychicfangs', 'thunderfang', 'firefang'], 'baseAbility': 'strongjaw', 'item': 'lifeorb', 'pokeball': 'pokeball', 'ability': 'strongjaw'}
bulkup: bul

spe:233
Ability: defeatist
Moves
{'ident': 'p1: Archeops', 'details': 'Archeops, L84, M', 'condition': '80/263', 'active': True, 'stats': {'atk': 283, 'def': 157, 'spa': 236, 'spd': 157, 'spe': 233}, 'moves': ['earthquake', 'uturn', 'stoneedge', 'dualwingbeat'], 'baseAbility': 'defeatist', 'item': 'choiceband', 'pokeball': 'pokeball', 'ability': 'defeatist'}
earthquake: earthquake (Move object)
uturn: uturn (Move object)
stoneedge: stoneedge (Move object)
dualwingbeat: dualwingbeat (Move object)
None
Effects: []
Item: choiceband
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 84
atk:283
def:157
spa:236
spd:157
spe:233
Ability: defeatist
Moves
{'ident': 'p1: Archeops', 'details': 'Archeops, L84, M', 'condition': '80/263', 'active': True, 'stats': {'atk': 283, 'def': 157, 'spa': 236, 'spd': 157, 'spe': 233}, 'moves': ['earthquake', 'uturn', 'stoneedge', 'dualwingbeat'], 'baseAbility': 'defeatist', 'item': 'choiceband', 'pokeball': 'pokeball', 'ability': 'defeatist'}
eart

spa:146
spd:180
spe:203
Ability: guts
Moves
{'ident': 'p1: Obstagoon', 'details': 'Obstagoon, L82, M', 'condition': '252/287', 'active': True, 'stats': {'atk': 195, 'def': 213, 'spa': 146, 'spd': 180, 'spe': 203}, 'moves': ['bulkup', 'facade', 'knockoff', 'closecombat'], 'baseAbility': 'guts', 'item': 'flameorb', 'pokeball': 'pokeball', 'ability': 'guts'}
bulkup: bulkup (Move object)
facade: facade (Move object)
knockoff: knockoff (Move object)
closecombat: closecombat (Move object)
None
Effects: []
Item: flameorb
accuracy: 0
atk: 0
def: 0
evasion: 0
spa: 0
spd: 0
spe: 0
level: 82
atk:195
def:213
spa:146
spd:180
spe:203
Ability: guts
Moves
{'ident': 'p1: Obstagoon', 'details': 'Obstagoon, L82, M', 'condition': '252/287 brn', 'active': True, 'stats': {'atk': 195, 'def': 213, 'spa': 146, 'spd': 180, 'spe': 203}, 'moves': ['bulkup', 'facade', 'knockoff', 'closecombat'], 'baseAbility': 'guts', 'item': 'flameorb', 'pokeball': 'pokeball', 'ability': 'guts'}
bulkup: bulkup (Move object)
facad

2021-01-09 22:00:49,506 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,507 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,508 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,509 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,509 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,510 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,511 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,512 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,513 - RandomPlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,514 - MaxDamagePlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,515 - MaxDamagePlayer 2 - CRITICAL - CancelledError intercepted: 
2021-01-09 22:00:49,515 - MaxDamagePlayer 2 - CRITICAL - CancelledError intercept

### OpenAI Gym Wrapper

State Space: base power / damage multiplier / pokemon left / opponent pokemon left

A 1d tensor of length 10

Action Space: Given by agent

In [ ]:
from poke_env.player.env_player import Gen8EnvSinglePlayer
import numpy as np


class SimpleRLPlayer(Gen8EnvSinglePlayer):
    def embed_battle(self, battle):
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Normalize 0~1
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                )

        remaining_mon_team = (
            len([mon for mon in battle.team.values() if mon.fainted]) / 6
        )
        remaining_mon_opponent = (
            len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6
        )

        # Final vector with 10 components
        return np.concatenate(
            [
                moves_base_power,
                moves_dmg_multiplier,
                [remaining_mon_team, remaining_mon_opponent],
            ]
        )

    def compute_reward(self, battle) -> float:
        return self.reward_computing_helper(
            battle, fainted_value=2, hp_value=1, victory_value=30
        )

Instantiate the player

In [ ]:
env_player = SimpleRLPlayer(battle_format="gen8randombattle")

In [ ]:
env_player.action_space

Now, build a deep q network

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

n_action = len(env_player.action_space)

model = Sequential()
model.add(Dense(128, activation="elu", input_shape=(1, 10,)))
model.add(Flatten())
model.add(Dense(64, activation="elu"))
model.add(Dense(n_action, activation="linear"))

Define DQN

In [ ]:
from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from tensorflow.keras.optimizers import Adam

memory = SequentialMemory(limit=10000, window_length=1)

policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr="eps",
    value_max=1.0,
    value_min=0.05,
    value_test=0,
    nb_steps=10000,
)

# Defining our DQN
dqn = DQNAgent(
    model=model,
    nb_actions=len(env_player.action_space),
    policy=policy,
    memory=memory,
    nb_steps_warmup=1000,
    gamma=0.5,
    target_model_update=1,
    delta_clip=0.01,
    enable_double_dqn=True,
)

dqn.compile(Adam(lr=0.00025), metrics=["mae"])

Set Hyperparameters

In [ ]:
NB_TRAINING_STEPS = 100
NB_EVALUATION_EPISODES = 100

Functions needed

In [ ]:
from threading import Thread
from poke_env.utils import to_id_str


def dqn_training(player, dqn, nb_steps):
    dqn.fit(player, nb_steps=nb_steps)
    print("Fit complete")
    player.complete_current_battle()
    print("Complete the rest")
    
def dqn_evaluation(player, dqn, nb_episodes):
    player.reset_battles()
    dqn.test(player, nb_episodes=nb_episodes, visualize=False, verbose=False)

    print(
        "DQN Evaluation: %d victories out of %d episodes"
        % (player.n_won_battles, nb_episodes)
    )

async def play_with(player, opponent):
    player._start_new_battle = True
    
    async def launch_battles():
        battles_coroutine = asyncio.gather(
            player.send_challenges(
                opponent=to_id_str(opponent.username),
                n_challenges=1,
                to_wait=opponent.logged_in,
            ),
            opponent.accept_challenges(
                opponent=to_id_str(player.username), n_challenges=1
            ),
        )
        await battles_coroutine
    
    def play():
        dqn_training(player=player, dqn=dqn, nb_steps=NB_TRAINING_STEPS)
        player._start_new_battle = False
        while True:
            try:
                player.complete_current_battle()
                player.reset()
            except OSError:
                break
    
    thread = Thread(target=play, daemon=True)
    thread.start()

    while player._start_new_battle:
        await launch_battles()

Start training

In [ ]:
opponent = RandomPlayer(battle_format="gen8randombattle")

await play_with(env_player, opponent)

Evaulating the agent

In [ ]:
def dqn_evaluation(player, dqn, nb_episodes):
    player.reset_battles()
    dqn.test(player, nb_episodes=nb_episodes, visualize=False, verbose=False)

    print(
        "DQN Evaluation: %d victories out of %d episodes"
        % (player.n_won_battles, nb_episodes)
    )

In [ ]:
def evalulate():
    dqn_evaluation(player=env_player, dqn=dqn, nb_steps=NB_TRAINING_STEPS)
    env_player._start_new_battle = False
    while True:
        try:
            env_player.complete_current_battle()
            env_player.reset()
        except OSError:
            break
            
thread = Thread(target=play, daemon=True)
thread.start()

while env_player._start_new_battle:
    await launch_battles()

### Connecting to Showdown Official


In [ ]:
from poke_env.player_configuration import PlayerConfiguration
from poke_env.server_configuration import ShowdownServerConfiguration

ID: GokemonRox

PW: gokemon

이라는 showdown 계정을 만들었다. 위 계정을 이용하여 접속 가능.

In [ ]:
player = MaxDamagePlayer(
    player_configuration=PlayerConfiguration("GokemonRox", "gokemon"),
    server_configuration=ShowdownServerConfiguration
)

만약 이 agent 와 한국어로 배틀을 하고 싶다면, 사설 서버인 포다운을 이용해서 배틀을 한다.

[포다운 다이렉트 서버](https://play.podown.pro/?p)

위 주소로 접속해서 본인 아이디로 로그인을 하고
GokemonRox 계정에 도전을 한다. (별도의 세팅이 없을 시 8세대 랜덤배틀)

이후 아래 코드로 도전을 수락한다.

In [ ]:
await player.accept_challenges(None, 1)